## Pre-processing of CRU climatic data

In [1]:
import xarray as xr
import numpy as np

import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [25]:
vap = xr.open_dataset('./Data/CRU/cru_ts4.00.1901.2015.vap.dat.nc') # hPa
tmp = xr.open_dataset('./Data/CRU/cru_ts4.00.1901.2015.tmp.dat.nc') # °C
pet = xr.open_dataset('./Data/CRU/cru_ts4.00.1901.2015.pet.dat.nc') # mm/day
tmx = xr.open_dataset('./Data/CRU/cru_ts4.00.1901.2015.tmx.dat.nc') # °C
pre = xr.open_dataset('./Data/CRU/cru_ts4.00.1901.2015.pre.dat.nc') # mm/month

### locations of selected sites

In [31]:
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [32]:
loc = pd.read_excel('./Data/Tree_Ring/site_info.xlsx',index_col=0)

In [41]:
vap_all = []
tmp_all = []
pet_all = []
tmx_all = []
pre_all = []

for site in loc.index:
    lat = loc[loc.index==site].Lat.values[0]
    lon = loc[loc.index==site].Lon.values[0]

    lat_nearest = find_nearest(vap.lat.values,lat)
    lon_nearest = find_nearest(vap.lon.values,lon)

    vap_site = vap.sel(lat=lat_nearest, lon=lon_nearest).vap.values#[:,None]
    tmp_site = tmp.sel(lat=lat_nearest, lon=lon_nearest).tmp.values#[:,None]
    pet_site = pet.sel(lat=lat_nearest, lon=lon_nearest).pet.values#[:,None]
    tmx_site = tmx.sel(lat=lat_nearest, lon=lon_nearest).tmx.values#[:,None]
    pre_site = pre.sel(lat=lat_nearest, lon=lon_nearest).pre.values#[:,None]

    vap_all.append(vap_site)
    tmp_all.append(tmp_site)
    pet_all.append(pet_site)
    tmx_all.append(tmx_site)
    pre_all.append(pre_site)

vap_all = np.array(vap_all).transpose()
tmp_all = np.array(tmp_all).transpose()
pet_all = np.array(pet_all).transpose()
tmx_all = np.array(tmx_all).transpose()
pre_all = np.array(pre_all).transpose()

In [44]:
date = pd.date_range('1901-01','2016-01',freq='M')

vap_all = pd.DataFrame(vap_all,columns=loc.index,index=date)
tmp_all = pd.DataFrame(tmp_all,columns=loc.index,index=date)
pet_all = pd.DataFrame(pet_all,columns=loc.index,index=date)
tmx_all = pd.DataFrame(tmx_all,columns=loc.index,index=date)
pre_all = pd.DataFrame(pre_all,columns=loc.index,index=date)

# vap_all = vap_all.set_index(date)
# tmp_all = tmp_all.set_index(date)

### calculate relative humidity and VPD

In [7]:
# compute saturation vapor pressure

def saturation_vapor_pressure(temp):
    '''
    To compute the saturation vapor pressure at specified temperature(°C)
    
    Input:
    temp: temperature in °C
    
    Output:
    es: saturation vapor pressure in hPa
    
    '''
    temp = temp + 273.15 # °C --> °K
    es = 611*np.exp(2.5*10**6/461*(1/273.16-1/temp)) # Pa
    es = es/100 # Pa --> hPa
    
    return es


def relative_humidity(temp,vap):
    
    '''
    To compute the relative humidity at specified temperature and water vapor pressure
    
    Input:
    temp: temperature in °C
    vap:  water vapor pressure in hPa
    
    Output:
    RH: relative humidity (%)
    '''
    
    es = saturation_vapor_pressure(temp)
    RH = vap/es * 100
    
    return RH

def vapor_pressure_deficit(temp,vap):
    
    '''
    To compute the vapor pressure deficit at specified temperature and water vapor pressure
    
    Input:
    temp: temperature in °C
    vap:  water vapor pressure in hPa
    
    Output:
    VPD: vapor pressure deficit in hPa
    '''
    
    es  = saturation_vapor_pressure(temp)
    VPD = es - vap
    
    return VPD

In [8]:
RH_all  = relative_humidity(tmp_all,vap_all) 
VPD_all = vapor_pressure_deficit(tmp_all,vap_all) # hPa

In [48]:
tmp_all.to_csv('./Data/CRU/Temp_monthly_all_sites.csv')
VPD_all.to_csv('./Data/CRU/VPD_monthly_all_sites.csv')
RH_all.to_csv('./Data/CRU/RH_monthly_all_sites.csv')
pet_all.to_csv('./Data/CRU/Pet_monthly_all_sites.csv')
tmx_all.to_csv('./Data/CRU/Tmx_monthly_all_sites.csv')
pre_all.to_csv('./Data/CRU/Pre_monthly_all_sites.csv')

Select growing season period and calculate the annual average value 

Growing season: May - Sep

In [56]:
RH_all['Year'] = RH_all.index.year
RH_all_GS = RH_all[(RH_all.index.month>=5)&(RH_all.index.month<=9)]
RH_annual = RH_all_GS.groupby('Year').mean()

In [57]:
VPD_all['Year'] = VPD_all.index.year
VPD_all_GS = VPD_all[(VPD_all.index.month>=5)&(VPD_all.index.month<=9)]
VPD_annual = VPD_all_GS.groupby('Year').mean()

In [53]:
tmp_all['Year'] = tmp_all.index.year
tmp_all_GS = tmp_all[(tmp_all.index.month>=5)&(tmp_all.index.month<=9)]
tmp_annual = tmp_all_GS.groupby('Year').mean()

In [49]:
pet_all['Year'] = pet_all.index.year
pet_all_GS = pet_all[(pet_all.index.month>=5)&(pet_all.index.month<=9)]
pet_annual = pet_all_GS.groupby('Year').mean()

In [50]:
tmx_all['Year'] = tmx_all.index.year
tmx_all_GS = tmx_all[(tmx_all.index.month>=5)&(tmx_all.index.month<=9)]
tmx_annual = tmx_all_GS.groupby('Year').mean()

In [55]:
pre_all['Year'] = pre_all.index.year
pre_all_GS = pre_all[(pre_all.index.month>=5)&(pre_all.index.month<=9)]
pre_annual = pre_all_GS.groupby('Year').mean()

In [58]:
tmp_annual.to_csv('./Data/CRU/Temp_all_sites.csv')
VPD_annual.to_csv('./Data/CRU/VPD_all_sites.csv')
RH_annual.to_csv('./Data/CRU/RH_all_sites.csv')
tmx_annual.to_csv('./Data/CRU/Tmx_all_sites.csv')
pet_annual.to_csv('./Data/CRU/Pet_all_sites.csv')
pre_annual.to_csv('./Data/CRU/Pre_all_sites.csv')